In [ ]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import img_to_array

# Load models
harassment_model_path = "harass.h5"
harassment_model = tf.keras.models.load_model(harassment_model_path)

gender_model_path = "gender_classification_model.h5"
gender_model = tf.keras.models.load_model(gender_model_path)

expression_model_path = "emotion_model.h5"
expression_model = tf.keras.models.load_model(expression_model_path)

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to process frames for harassment detection
def process_frame_for_harassment(frame):
    img = cv2.resize(frame, (224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    features = features.reshape(1, 7 * 7 * 512)
    predictions = harassment_model.predict(features)
    return np.argmax(predictions[0])

# Function to process frames for gender detection
def process_frame_for_gender(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    male_count = 0
    female_count = 0
    lone_woman_detected = False

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (64, 64))
        roi_array = img_to_array(roi_resized) / 255.0
        roi_array = np.expand_dims(roi_array, axis=0)
        
        prediction = gender_model.predict(roi_array)
        result = "Male" if prediction[0][0] >= 0.5 else "Female"
        
        if result == "Male":
            male_count += 1
        else:
            female_count += 1
            if male_count == 0:
                lone_woman_detected = True
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, result, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, male_count, female_count, lone_woman_detected

# Function to process frames for emotion detection
def process_frame_for_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    detected_emotions = []

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (48, 48))
        face_roi_normalized = face_roi_resized / 255.0
        face_roi_normalized = face_roi_normalized.astype(np.float32)
        face_roi_rgb = cv2.cvtColor(face_roi_normalized, cv2.COLOR_GRAY2RGB)
        face_roi_expanded = np.expand_dims(face_roi_rgb, axis=0)
        
        emotion_prediction = expression_model.predict(face_roi_expanded)
        emotion_label = emotion_labels[np.argmax(emotion_prediction)]
        detected_emotions.append(emotion_label)
        
        # Emotion label below the bounding box
        cv2.putText(frame, emotion_label, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    return frame, detected_emotions

# Streamlit app
def main():
    st.title("Real-Time Detection: Harassment, Gender, and Emotion")
    
    # Video input using Streamlit camera
    use_camera = st.checkbox("Use Camera for Video Feed", value=True)
    
    if use_camera:
        video_capture = cv2.VideoCapture(0)
        st.sidebar.text("Press 'Stop' to end the video.")
        frame_window = st.image([])

        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                st.error("Unable to access camera feed.")
                break

            harassment_prediction = process_frame_for_harassment(frame)
            frame, male_count, female_count, lone_woman_detected = process_frame_for_gender(frame)
            frame, detected_emotions = process_frame_for_emotion(frame)

            # Overlay detection results on the frame
            if harassment_prediction == 1:
                cv2.putText(frame, "Harassment Detected!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            if lone_woman_detected:
                cv2.putText(frame, "Lone Woman Detected", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

            # Display the video frame
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert color for Streamlit
            frame_window.image(frame)

        video_capture.release()

if __name__ == "__main__":
    main()
